In [1]:
import numpy
import pandas as pd
import requests
import nba_api
import pathlib as plib
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playergamelog


In [2]:
player_list = [
        '1631101' #Sharpe
        ,'1629014' #Simons
        ,'203924' #Grant
        ,'1629234' #Eubanks
        ,'1628404' #Hart
        ,'203994' #Nurkic
        ,'203081' #Lillard
        ,'1630570' #Watford
        ,'1631133' #Walker
        ,'1629642' #Little
        ,'1626159' #Winslow
        ,'1630553' #Johnson
        ,'1630535' #Brown III
        ,'1627780' #Payton II
        ]

In [3]:
df1 = pd.DataFrame()

for playerid in player_list:
        player_list = playerid
        player_data = playergamelog.PlayerGameLog(player_id=[playerid])
        df = player_data.get_data_frames()[0]
        df1 = pd.concat([df1,df], ignore_index=True, sort=False)##Combines Sequential
#career.append(playergamelog.PlayerGameLog(player_id={203081}))
#df = career.get_data_frames()[0]

In [4]:


filepath = plib.Path(r'C:\Users\Danny\git_projects\outputs\sports_analysis_outputs\playerstats.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

df1.to_csv(filepath)     

In [5]:
print("dope")

dope
